In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_4_1a.csv",
    index_col=0,
)
config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 2000.,
    "t_step": 1.,
}
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
model = CompartmentalModel(
    times=(0, config["end_time"]),
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=config["t_step"],
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infectious_seed"],
        "Infectious": config["infectious_seed"]
    }
)
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("r0") / ave_infous,
    source="Susceptible",
    dest="Pre-infectious"
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
parameters = {
    "r0": 2.,
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "life_expectancy": 70.,
}
model.run(parameters=parameters)
seir_values = model.get_outputs_df()
assert abs(seir_data - seir_values).max().max() < 1e5